In [ ]:
pip install opencv-python ultralytics numpy

In [ ]:
!pip install opencv-python ultralytics numpy sqlite3

In [ ]:
pip install mysql-connector-python

In [1]:
import cv2
import sqlite3
from datetime import datetime
from ultralytics import YOLO

# Load YOLOv5 model (or YOLOv8)
model = YOLO('yolov5s.pt')  # Automatically downloads if not present

# Connect to SQLite database
conn = sqlite3.connect('people_tracking.db')
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS people_visits (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT,
        count INTEGER
    )
""")
conn.commit()

# Open video source (0 for default webcam)
video_source = 0
cap = cv2.VideoCapture(video_source)

print("🔍 Press 'q' to quit")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLO detection
        results = model(frame, stream=True)
        people_count = 0

        for r in results:
            for box in r.boxes:
                cls = int(box.cls[0])
                label = model.names[cls]
                if label == "person":
                    people_count += 1
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Display count
        cv2.putText(frame, f"People Count: {people_count}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Store count with timestamp
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cursor.execute("INSERT INTO people_visits (timestamp, count) VALUES (?, ?)", (timestamp, people_count))
        conn.commit()

        cv2.imshow("YOLO People Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("\n⛔ Interrupted by user")

finally:
    cap.release()
    conn.close()
    cv2.destroyAllWindows()
    print("✅ Application closed cleanly")

PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.

🔍 Press 'q' to quit

0: 480x640 2 persons, 188.3ms
Speed: 7.1ms preprocess, 188.3ms inference, 9.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 126.7ms
Speed: 3.7ms preprocess, 126.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 103.5ms
Speed: 1.3ms preprocess, 103.5ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 105.1ms
Speed: 1.9ms preprocess, 105.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 101.6ms
Speed: 1.9ms preprocess, 101.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 108.1ms
Speed: 1.8ms preprocess, 108.1ms inference, 0.9m